# Dog breed multi-class classifier

> Building a multi class image classifier with tensorflow.

## Problem

> Identifying a breed of a dog given an image of a dog.

## Data

> Since this problem is based around unstructured data, the solution would be to use deep learning/transfer learning, the data can be found [here](https://www.kaggle.com/c/dog-breed-identification/data?select=sample_submission.csv).

In [3]:
import tensorflow as tf
import tensorflow_hub as hub

#check for GPU
print(f'GPU: {len(tf.config.list_physical_devices("GPU"))}')

GPU: 1
